Using the given python script to generate a cutout of NGC3379 and convert the inverse variance map to sigma map

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

# -------------------------------------------------
# Input brick files (you must edit these paths)
# -------------------------------------------------
image_file  = "/mnt/c/Users/lenovo/Desktop/FRB-capstone/legacysurvey-1619p125-image-r.fits.fz"
invvar_file = "/mnt/c/Users/lenovo/Desktop/FRB-capstone/legacysurvey-1619p125-invvar-r.fits.fz"


# -------------------------------------------------
# Target coordinates: NGC 3379 (M105)
# -------------------------------------------------
center = SkyCoord(ra=161.9567 * u.deg, dec=12.5817 * u.deg, frame="icrs")

# Cutout size on the sky
size = (200 * u.arcsec, 200 * u.arcsec)  # adjust if needed

# Output filenames
out_sci   = "NGC3379_r_cutout.fits"
out_sigma = "NGC3379_r_sigma_cutout.fits"

# -------------------------------------------------
# Read the brick images (note: data stored in HDU 1)
# -------------------------------------------------
with fits.open(image_file) as hdul_img, fits.open(invvar_file) as hdul_inv:
    img_hdu = hdul_img[1]      # science image
    inv_hdu = hdul_inv[1]      # inverse variance image

    data_img = img_hdu.data
    data_inv = inv_hdu.data

    # WCS from the science header
    wcs_full = WCS(img_hdu.header)

    # -------------------------------------------------
    # 1. Make science and invvar cutouts
    # -------------------------------------------------
    cutout_img = Cutout2D(data_img, position=center, size=size, wcs=wcs_full)
    cutout_inv = Cutout2D(data_inv, position=center, size=size, wcs=wcs_full)

    # -------------------------------------------------
    # 2. Convert invvar → sigma map
    #    invvar = 1/sigma^2  => sigma = 1/sqrt(invvar)
    # -------------------------------------------------
    inv = cutout_inv.data
    sigma = np.full(inv.shape, 1e9, dtype=float) # safer default for zero invvar
    mask = inv > 0
    sigma[mask] = 1.0 / np.sqrt(inv[mask])

    # -------------------------------------------------
    # 3. Write output FITS files with WCS
    # -------------------------------------------------
    hdr_cut = cutout_img.wcs.to_header()

    # fits.PrimaryHDU(data=cutout_img.data, header=hdr_cut).writeto(out_sci, overwrite=True)
    fits.PrimaryHDU(data=sigma, header=hdr_cut).writeto(out_sigma, overwrite=True)

print("Wrote:", out_sci)
print("Wrote:", out_sigma)

Wrote: NGC3379_r_cutout.fits
Wrote: NGC3379_r_sigma_cutout.fits


In [7]:
import numpy as np
from astropy.io import fits

# --- CONFIGURATION ---
input_filename = 'NGC3379_r_nan_sigma_cutout.fits'  # Your input file with NaNs
output_filename = 'NGC3379_r_sigma_FIXED.fits'     # The new file to create
fill_value = 1e9                                     # The value to replace NaNs with

# --- PROCESSING ---
print(f"Reading {input_filename}...")

try:
    # Open the FITS file
    with fits.open(input_filename) as hdul:
        # Load the data. usually in the primary HDU (index 0) for simple cutouts
        # If your cutout script put it in extension 1, change this to hdul[1].data
        data = hdul[0].data.astype(np.float32)
        header = hdul[0].header

        # Check for NaNs before fixing
        nan_count = np.count_nonzero(np.isnan(data))
        print(f"Found {nan_count} pixels with NaN values.")

        # REPLACE NaNs with the fill value
        # We use np.nan_to_num for safety, or direct boolean indexing
        data[np.isnan(data)] = fill_value
        
        # OPTIONAL: Also replace <= 0 values if any exist (physically impossible for sigma)
        zero_count = np.count_nonzero(data <= 0)
        if zero_count > 0:
            print(f"Found {zero_count} pixels with <= 0 values. Fixing those too.")
            data[data <= 0] = fill_value

        # Save the new file
        print(f"Writing corrected data to {output_filename}...")
        fits.PrimaryHDU(data=data, header=header).writeto(output_filename, overwrite=True)

    print("Done! Use this new file in your GALFIT feedme (Line C).")

except FileNotFoundError:
    print(f"Error: Could not find file '{input_filename}'. Check the name.")
except Exception as e:
    print(f"An error occurred: {e}")

Reading NGC3379_r_nan_sigma_cutout.fits...
Found 423 pixels with NaN values.
Writing corrected data to NGC3379_r_sigma_FIXED.fits...
Done! Use this new file in your GALFIT feedme (Line C).


In [9]:
import numpy as np
from astropy.io import fits

# --- CONFIGURATION ---
# Use the file that definitely has NaNs or Infinities in the center
input_sigma_file = 'NGC3379_r_nan_sigma_cutout.fits' 
output_mask_file = 'bad_pixel_mask.fits'

print(f"Reading {input_sigma_file}...")

try:
    with fits.open(input_sigma_file) as hdul:
        # Load data (usually extension 0 or 1, check your file structure)
        data = hdul[0].data
        header = hdul[0].header

        # Initialize mask with 0 (Good pixels)
        # GALFIT convention: 0 = Good, >0 = Bad
        mask = np.zeros_like(data, dtype=np.int16)

        # Flag NaNs
        nan_count = np.count_nonzero(np.isnan(data))
        mask[np.isnan(data)] = 1
        print(f"Flagged {nan_count} NaN pixels.")

        # Flag Infinities (just in case)
        inf_count = np.count_nonzero(np.isinf(data))
        mask[np.isinf(data)] = 1
        print(f"Flagged {inf_count} Infinite pixels.")
        
        # Optional: Flag Zero/Negative Sigma (Physical impossibility = Bad data)
        zero_count = np.count_nonzero(data <= 0)
        # Don't double count NaNs (comparisons with NaN are tricky)
        # Use a safe mask for this check
        valid_data_mask = ~np.isnan(data) & ~np.isinf(data)
        zeros_to_flag = (data <= 0) & valid_data_mask
        mask[zeros_to_flag] = 1
        print(f"Flagged {np.count_nonzero(zeros_to_flag)} Zero/Negative pixels.")

        # Save the mask
        print(f"Saving mask to {output_mask_file}...")
        fits.PrimaryHDU(data=mask, header=header).writeto(output_mask_file, overwrite=True)

    print("Done! Update 'Line F' in your galfit.feedme file.")

except FileNotFoundError:
    print(f"Error: Could not find {input_sigma_file}")
except Exception as e:
    print(f"An error occurred: {e}")

Reading NGC3379_r_nan_sigma_cutout.fits...
Flagged 423 NaN pixels.
Flagged 0 Infinite pixels.
Flagged 0 Zero/Negative pixels.
Saving mask to bad_pixel_mask.fits...
Done! Update 'Line F' in your galfit.feedme file.


Using Sextractor to generate a binary fits file, with the information of isolated point stars and small image cutouts of these stars ( 35 by 35 ) pixels for now, which will be fed to psfex.

In [ ]:
# edited default.sex to correct arcsec pixel ratio, to increase signal noise ratio(detect thresh) and to change file type of output
# edited default.param to only contain values of interest and include 35by35 pixel cutouts for psfex
# had to bring default.conv and default.nnw to current directory
# ran sex legacysurvey-1619p125-image-r.fits.fz -c default.sex -CATALOG_NAME ForPSF.cat

We verify that the CLASS_STAR is upheld through this python script. 

In [ ]:
from astropy.io import fits
from astropy.table import Table
import numpy as np

# Define your strict cutoff
MIN_CLASS_STAR = 0.8  # The guideline from your PDF

catalog_file = 'ForPSF.cat'
clean_file = 'clean_stars.cat'

print(f"--- CLASS_STAR CLEANER: {catalog_file} ---")

# Load the catalog (Data is usually in HDU 2 for LDAC)
try:
    t = Table.read(catalog_file, hdu=2)
except FileNotFoundError:
    print(f"Error: {catalog_file} not found!")
    exit()

total_objects = len(t)
print(f"Total Objects Detected: {total_objects}")

# ---------------------------------------------------------
# FILTER ONLY BY CLASS_STAR
# We are ignoring SNR, Ellipticity, and Flags for now.
# ---------------------------------------------------------
final_mask = t['CLASS_STAR'] >= MIN_CLASS_STAR
clean_table = t[final_mask]

cleaned_count = len(clean_table)
removed_count = total_objects - cleaned_count

print(f"Objects with CLASS_STAR >= {MIN_CLASS_STAR}: {cleaned_count}")
print(f"Objects removed (CLASS_STAR < {MIN_CLASS_STAR}): {removed_count}")

# ---------------------------------------------------------
# Save as a new FITS_LDAC file
# ---------------------------------------------------------
# We need to preserve the HDU structure for PSFEx to read it
hdul = fits.open(catalog_file)
hdul[2].data = clean_table.as_array() # Replace data with filtered version
hdul.writeto(clean_file, overwrite=True)
hdul.close()

print("-" * 50)
print(f"✅ CLEANED CATALOG SAVED: {clean_file}")
print(f"   Contains {len(clean_table)} guaranteed stars.")
print(f"   Action: Run 'psfex {clean_file}' to build your model.")
print("-" * 50)

--- CLASS_STAR CLEANER: ForPSF.cat ---
Error: ForPSF.cat not found!


NameError: name 't' is not defined

: 

We now use PSFEx to generate a model of the telescope blur ( Point Spread Function ), we hope to find. We attempt FLAG = 0.

In [ ]:
# Changed snr > 30 over 20, Max_ellp = .2 over .3, and set FLAG to only include perfect stars
# ran psfex ForPSF.cat
# Analyzed the various statistic fits files

We now attempt to generate parameters through galfit

In [3]:
from astropy.io import fits

# Open the prototype file
# Note: Usually the data is in extension 0 or 1.
# PSFEx saves the basis images in a 3D cube or a 2D strip.
hdul = fits.open('proto_clean_stars.fits') # Or whatever your proto file is named
data = hdul[0].data

# Check dimensions
print(f"Original Shape: {data.shape}")

# If it's a strip (e.g., 25 pixels tall, 525 pixels wide)
# We usually want the FIRST 25x25 box.
# Adjust '25' to match your PSF_SIZE from default.psfex
psf_size = 25 
psf_kernel = data[0:psf_size, 0:psf_size]

# Save it as the final PSF for GALFIT
fits.PrimaryHDU(data=psf_kernel).writeto('final_psf.fits', overwrite=True)
print("Saved 'final_psf.fits'. Use THIS in GALFIT.")

Original Shape: (25, 150)
Saved 'final_psf.fits'. Use THIS in GALFIT.
